## Import Library

In [4]:
# !pip install awswrangler -q

In [3]:
import awswrangler as wr
import numpy as np
import pandas as pd
import json
import gc

## Load Data

In [4]:
## Change Data Format
# monitor_hist = wr.s3.read_csv('s3://dd-ai-ml/raw_data/monitoring_history_202304241141.csv')
# monitor_hist.to_parquet('s3://dd-ai-ml/preprocessing/monitoring_history_202304241141.parquet')
# monitor_hist = wr.s3.read_parquet('s3://dd-ai-ml/preprocessing/monitoring_history_202304241141.parquet')
# monitor_hist = pd.json_normalize(monitor_hist.data.apply(json.loads))
# monitor_hist.to_csv('s3://dd-ai-ml/raw_data/preprocessing/parse_data.csv', index = False)

In [5]:
monitor_hist = wr.s3.read_csv('s3://dd-ai-ml/raw_data/preprocessing/parse_data.csv')
control_setting = wr.s3.read_csv('s3://dd-ai-ml/raw_data/control_setting_202304241113.csv')
controller = wr.s3.read_csv('s3://dd-ai-ml/raw_data/controller_202304241123.csv')
device = wr.s3.read_csv('s3://dd-ai-ml/raw_data/device_202304241047.csv')
device_log = wr.s3.read_csv('s3://dd-ai-ml/raw_data/device_control_log_202304241041.csv')
gsm_info = wr.s3.read_csv('s3://dd-ai-ml/raw_data/gsm_info_202304241114.csv') ## 농가 정보

print(f'monitor_hist : {monitor_hist.shape}')
print(f'control_setting : {control_setting.shape}')
print(f'controller : {controller.shape}')
print(f'device : {device.shape}')
print(f'device_log : {device_log.shape}')
print(f'gsm_info : {gsm_info.shape}')

monitor_hist : (902910, 59)
control_setting : (7, 15)
controller : (6, 10)
device : (28, 29)
device_log : (214585, 15)
gsm_info : (379, 37)


In [14]:
device_log.head()

,seq,device_id,event_date,command,duration,actor,client_ip,reason,modbus_address1,modbus_address2,modbus_address3,run_command,block_reason,gsm_key,green_house_id
0,18333577,6345000000001,2021-04-22 01:58:57,Stop,0,99616269,175.202.71.114,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
1,18333578,6345000000001,2021-04-22 01:58:57,Stop,0,99616269,175.202.71.114,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
2,18333579,6345000000001,2021-04-22 01:58:57,Close,40,99616269,175.202.71.114,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
3,18333583,6345000000002,2021-04-22 01:59:09,Stop,0,99616269,175.202.71.114,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
4,18333584,6345000000002,2021-04-22 01:59:09,Stop,0,99616269,175.202.71.114,NaN,NaN,NaN,NaN,1,NaN,6345,NaN


In [10]:
control_setting

,id,gsm_key,green_house_id,logic_name,logic_id,logic_period_size,logic_env,logic_period_env,create_dt,update_dt,auto_manual_mode,java_class_name,control_status,view_order,update_system
0,6345000000001,6345,6345000000001,1중창,56,0,"{""close_temperature"":null,""condition"":null,""ma...",[],2020-12-17 11:23:22,2022-09-22 08:31:41,auto,WindowRainSgController,NaN,1,supervisor
1,6345000000002,6345,6345000000001,보온-상,60,0,"{""condition"":null,""max_position"":null,""relativ...","[{""condition"":""3"",""relative_time"":""2020-12-16T...",2020-12-17 11:23:22,2022-07-10 05:52:36,manual,DCTimeController,NaN,4,supervisor
2,6345000000003,6345,6345000000001,보온-하,60,0,"{""condition"":null,""max_position"":null,""relativ...","[{""condition"":""3"",""relative_time"":""2020-12-16T...",2020-12-17 11:23:22,2022-04-19 14:09:22,manual,DCTimeController,NaN,5,supervisor
3,6345000000004,6345,6345000000001,2중창-하,59,0,"{""close_temperature"":null,""condition"":null,""ma...","[{""condition"":""1"",""relative_time"":""2020-12-16T...",2020-12-17 11:23:23,2022-04-19 15:06:08,manual,DCTempCondControlller,NaN,3,supervisor
4,6345000000005,6345,6345000000001,2중창-상,59,0,"{""close_temperature"":null,""condition"":null,""ma...","[{""condition"":""2"",""relative_time"":""2020-12-16T...",2020-12-17 11:23:23,2022-04-20 12:14:02,manual,DCTempCondControlller,NaN,2,supervisor
5,6345000000006,6345,6345000000001,유동팬,14,0,"{""condition"":null,""operation_hum"":null,""operat...","[{""condition"":""3"",""relative_start_time"":""2020-...",2020-12-17 11:23:23,2022-09-29 16:38:49,auto,FloatFanController,NaN,6,supervisor
6,6345000000007,6345,6345000000001,천창팬,11,0,"{""condition"":null,""operation_temperature"":null...","[{""period"":1,""condition"":""3"",""relative_start_t...",2020-12-17 11:23:23,2022-04-19 15:06:08,manual,ExhaustFanController,NaN,7,supervisor


## Data Filltering on gsm_key

In [5]:
## gsm_key == 6345 (정읍_박인식)
monitor_hist = monitor_hist[monitor_hist['gsm_key'] == 6345]
control_setting = control_setting[control_setting['gsm_key'] == 6345]
controller = controller[controller['gsm_key'] == 6345]
device = device[device['gsm_key'] == 6345]
device_log = device_log[device_log['gsm_key'] == 6345]
gsm_info = gsm_info[gsm_info['gsm_key'] == 6345]

print(f'monitor_hist : {monitor_hist.shape}')
print(f'control_setting : {control_setting.shape}')
print(f'controller : {controller.shape}')
print(f'device : {device.shape}')
print(f'device_log : {device_log.shape}')
print(f'gsm_info : {gsm_info.shape}')

monitor_hist : (902910, 59)
control_setting : (7, 15)
controller : (6, 10)
device : (28, 29)
device_log : (214585, 15)
gsm_info : (1, 37)


In [10]:
## 동일한 의미를 갖나, 표현하는 방식이 다름
print(control_setting['logic_name'].unique())
print(device['nickname'].unique())

['1중창' '보온-상' '보온-하' '2중창-하' '2중창-상' '유동팬' '천창팬']
['1중창[좌]' '1중창[우]' '2중창[상]' '2중창[하]' '제어기전원' '주전원' '보온[상]' '보온[하]' '유동팬'
 '천창팬' '온도1' '습도1' '지온1' '지습1' 'CO2-1' '온도2' '습도2' '지온2' '지습2' 'CO2-2'
 '감우' '광량' '습도' '외온' '풍속' '풍향' '평균온도' '평균습도']


## Delete All NaN Columns

In [11]:
def drop_all_nan(df):
    print(df.shape)
    for col in list(df):
        if df[col].isna().sum() == len(df):
            print(f'drop col : {col}')
            df.drop(col, axis = 1, inplace = True)
    print(f'{df.shape} \n')
    
drop_all_nan(monitor_hist)
drop_all_nan(control_setting)
drop_all_nan(controller)
drop_all_nan(device)
drop_all_nan(device_log)
drop_all_nan(gsm_info)

(902910, 59)
(902910, 59) 

(7, 15)
drop col : control_status
(7, 14) 

(6, 10)
drop col : port
drop col : controller_status
drop col : properties
(6, 7) 

(28, 29)
drop col : user_info_id
drop col : device_type_nickname
drop col : description
drop col : setup_date
drop col : memo
drop col : properties
drop col : reg_min
drop col : reg_max
drop col : min_value
drop col : max_value
(28, 19) 

(214585, 15)
drop col : modbus_address1
drop col : modbus_address2
drop col : modbus_address3
drop col : block_reason
drop col : green_house_id
(214585, 10) 

(1, 37)
drop col : owner_user_info_id
drop col : remote_address
drop col : package_version
drop col : schema_version
drop col : farm_nickname_i18n
drop col : user_id
drop col : farm_db_id
(1, 30) 



## Data Join

In [12]:
print(f'monitor_hist : {monitor_hist.shape}')
print(f'control_setting : {control_setting.shape}')
print(f'controller : {controller.shape}')
print(f'device : {device.shape}')
print(f'device_log : {device_log.shape}')
print(f'gsm_info : {gsm_info.shape}')

monitor_hist : (902910, 59)
control_setting : (7, 14)
controller : (6, 7)
device : (28, 19)
device_log : (214585, 10)
gsm_info : (1, 30)


In [40]:
merge_df = pd.merge(controller, gsm_info, on = 'gsm_key', how = 'left') ## join column unique 개수 같음
merge_df.rename(columns = {'update_date' : 'update_date_gsm'}, inplace = True)
merge_df.rename(columns = {'update_dt' : 'update_date_controller'}, inplace = True)
merge_df.rename(columns = {'id' : 'controller_id'}, inplace = True)
print(f'join controller & gsm_info shape : {merge_df.shape}')

merge_df = pd.merge(device, merge_df, on = ['controller_id', 'gsm_key'], how = 'left') ## join column unique 개수 같음
merge_df.rename(columns = {'update_date' : 'update_date_device'}, inplace = True)
merge_df.rename(columns = {'id' : 'device_id'}, inplace = True)
print(f'join device & merge_df shape : {merge_df.shape}')

merge_df = pd.merge(device_log, merge_df, on = ['gsm_key', 'device_id'], how = 'left')
merge_df.rename(columns = {'event_date' : 'event_date_log'}, inplace = True)
print(f'join device_log & merge_df shape : {merge_df.shape}')

join controller & gsm_info shape : (6, 36)
join device & merge_df shape : (28, 53)
join device_log & merge_df shape : (214585, 61)


## Explore Data

In [173]:
## 분석 대상 추출
print(control_setting['logic_name'].unique())
print(device['nickname'].unique())

['1중창' '보온-상' '보온-하' '2중창-하' '2중창-상' '유동팬' '천창팬']
['1중창[좌]' '1중창[우]' '2중창[상]' '2중창[하]' '제어기전원' '주전원' '보온[상]' '보온[하]' '유동팬'
 '천창팬' '온도1' '습도1' '지온1' '지습1' 'CO2-1' '온도2' '습도2' '지온2' '지습2' 'CO2-2'
 '감우' '광량' '습도' '외온' '풍속' '풍향' '평균온도' '평균습도']


In [110]:
## device_log의 unique device_id : 8, merge_df(device)의 unique device_id : 28개
print(device.id.nunique()) 
print(device_log.device_id.nunique())

28
8


In [209]:
# device

In [210]:
## device log에 포함되어있는 8개의 device id마다 존재하는 log 개수 다름
## 1~4까지 1중창[좌,우], 2중창 [상,하]의 log 개수가 전부 다름 (개별 모델링 필요)
for device_id in device_log.device_id.unique():    
    if str(device_id)[-1:] in ['1','2','3','4']:
        print(f"{device_id} device id has {device_log[device_log['device_id'] == device_id].shape[0]} row")

6345000000001 device id has 7473 row
6345000000002 device id has 6150 row
6345000000003 device id has 1084 row
6345000000004 device id has 84 row


In [213]:
## 동일한 대상 및 환경에 대해 중복되는 이벤트가 연속해서 발생 (2021-05-03 15:36:33 ~ 2021-05-03 15:46:51)
temp_df = device_log[(device_log['event_date'].str.split(' ',expand = True)[0] == '2021-05-03') & (device_log['device_id'] == 6345000000001) & (device_log['client_ip'] == '211.199.190.110')]
temp_df[temp_df['command'] != 'Stop'].head(12)

,seq,device_id,event_date,command,duration,actor,client_ip,reason,run_command,gsm_key
1604,19004439,6345000000001,2021-05-03 15:22:11,Open,60,99616269,211.199.190.110,NaN,1,6345
1607,19004452,6345000000001,2021-05-03 15:22:26,Close,100,99616269,211.199.190.110,NaN,1,6345
1610,19004498,6345000000001,2021-05-03 15:23:07,Open,60,99616269,211.199.190.110,NaN,1,6345
1613,19005152,6345000000001,2021-05-03 15:36:00,Open,60,99616269,211.199.190.110,NaN,1,6345
1618,19005173,6345000000001,2021-05-03 15:36:14,Close,99,99616269,211.199.190.110,NaN,1,6345
1623,19005192,6345000000001,2021-05-03 15:36:33,Open,60,99616269,211.199.190.110,NaN,1,6345
1626,19005225,6345000000001,2021-05-03 15:37:11,Open,60,99616269,211.199.190.110,NaN,1,6345
1637,19005271,6345000000001,2021-05-03 15:37:52,Open,60,99616269,211.199.190.110,NaN,1,6345
1640,19005292,6345000000001,2021-05-03 15:38:18,Open,60,99616269,211.199.190.110,NaN,1,6345
1643,19005714,6345000000001,2021-05-03 15:46:40,Open,60,99616269,211.199.190.110,NaN,1,6345


In [178]:
## 하나의 device id log에서 동일한 시간(event_date)에 다양한 command 존재
device_log[device_log['device_id'] == 6345000000001].head(3)

,seq,device_id,event_date,command,duration,actor,client_ip,reason,run_command,gsm_key
0,18333577,6345000000001,2021-04-22 01:58:57,Stop,0,99616269,175.202.71.114,NaN,1,6345
1,18333578,6345000000001,2021-04-22 01:58:57,Stop,0,99616269,175.202.71.114,NaN,1,6345
2,18333579,6345000000001,2021-04-22 01:58:57,Close,40,99616269,175.202.71.114,NaN,1,6345


In [197]:
# control_setting['logic_env'][0]

In [215]:
# device_log[(device_log['duration']>0) & (device_log['device_id'] == 6345000000001)].head(20)

## ERD 추가 설명 필요

In [176]:
## client_ip가 공백이 아닐경우 actor값 수치형으로 존재
## client_ip가 공백일 경우 actor값은 문자형 or NaN으로 존재
print(device_log[device_log['client_ip'].isna()]['actor'].unique())
print(device_log[~device_log['client_ip'].isna()]['actor'].unique())

['AutoControlSystem' 'Owner' nan]
['99616269' '99616386']


In [221]:
## device id 기준으로 device_log와 device_cont 매핑하려 하나 공통된 컬럼에 포함되어있는 정보가 너무 다름
## device_log 공통된 컬럼의 대부분 값이 nan으로 표현되어있음 -> device_cont 정보로 대체하는것이 맞는가?

for col in set(list(device_log)) & set(list(device)):
    print(f'device_log col {col} unique values : {device_log[col].unique()}')
    print(f'device col {col} unique values : {device[col].unique()}')
    print('\n')

device_log col gsm_key unique values : [6345]
device col gsm_key unique values : [6345]


device_log col modbus_address3 unique values : [nan]
device col modbus_address3 unique values : [ 2.  4.  6.  8. -1. nan]


device_log col modbus_address2 unique values : [nan]
device col modbus_address2 unique values : [ 1.  3.  5.  7. 10. 11.  6. nan]


device_log col green_house_id unique values : [nan]
device col green_house_id unique values : [0]


device_log col modbus_address1 unique values : [nan]
device col modbus_address1 unique values : ['1' '2' '3' '4' '5' '6' 'temp1' 'hum1' 'gtemp1' 'ghum1' 'co2' 'rain'
 'light' 'windsp' 'winddir' nan]




## Test

In [13]:
device['nickname'].unique()

array(['1중창[좌]', '1중창[우]', '2중창[상]', '2중창[하]', '제어기전원', '주전원', '보온[상]',
       '보온[하]', '유동팬', '천창팬', '온도1', '습도1', '지온1', '지습1', 'CO2-1', '온도2',
       '습도2', '지온2', '지습2', 'CO2-2', '감우', '광량', '습도', '외온', '풍속', '풍향',
       '평균온도', '평균습도'], dtype=object)

In [15]:
device[device['nickname'] == '평균온도'].values

array([[6345000000027, 20201, 1, nan, 6345, 0, 6345000000006, '평균온도',
        nan, nan, nan, 0, 0, 0, 0, nan, nan, nan, nan, 0, 1,
        '2021-10-14 10:02:59', 1, 1, nan, nan, nan, nan, nan]],
      dtype=object)

In [18]:
device_log[(device_log['duration'] != 0) & (device_log['device_id'] == 6345000000001)]['actor'].unique()

array(['99616269', 'AutoControlSystem', '99616386'], dtype=object)

In [21]:
device_log[(device_log['duration'] != 0) & (device_log['device_id'] == 6345000000001)]['reason'].unique()

array([nan, '자동 동작', '고온 위험 ,34.5도', '초기 설정', '고온 위험 ,33.3도',
       '고온 위험 ,25.6도', '고온 위험 ,31.3도', '고온 위험 ,26.1도', '고온 위험 ,21.1도',
       '감우 위험', '저온 위험, 18.0도', '저온 위험, 18.9도', '저온 위험, 17.0도',
       '저온 위험, 16.8도', '저온 위험, 16.5도', '저온 위험, 16.0도', '저온 위험, 15.5도',
       '고온 위험 ,21.7도', '저온 위험, 17.9도', '장비초기화', '저온 위험, 16.2도',
       '저온 위험, 16.3도', '저온 위험, 15.2도', '저온 위험, 14.5도', '저온 위험, 14.6도',
       '고온 위험 ,23.5도', '저온 위험, 17.8도', '저온 위험, 17.6도', '저온 위험, 17.4도',
       '저온 위험, 17.3도', '저온 위험, 17.1도', '저온 위험, 16.9도', '저온 위험, 16.6도',
       '저온 위험, 16.4도', '저온 위험, 14.7도', '저온 위험, 14.8도', '저온 위험, 14.9도',
       '저온 위험, 14.2도', '저온 위험, 14.1도', '저온 위험, 14.3도', '저온 위험, 14.4도',
       '저온 위험, 13.9도', '저온 위험, 14.0도', '저온 위험, 13.8도', '저온 위험, 13.7도',
       '저온 위험, 13.6도', '저온 위험, 13.5도', '저온 위험, 13.4도', '저온 위험, 16.7도',
       '저온 위험, 15.0도', '저온 위험, 15.3도', '저온 위험, 15.8도', '저온 위험, 17.2도',
       '저온 위험, 17.5도', '저온 위험, 16.1도', '저온 위험, 13.3도', '저온 위험, 13.2도',
       '저온 위험, 13.1도', '저온 위험

In [22]:
device_log[(device_log['duration'] != 0) & (device_log['device_id'] == 6345000000001)]['command'].unique()

array(['Close', 'Open', 'AutoInitOpen', 'AutoInitClose'], dtype=object)

In [25]:
device_log[(device_log['duration'] != 0) & (device_log['device_id'] == 6345000000001)& (device_log['command'] == 'Open')]

,seq,device_id,event_date,command,duration,actor,client_ip,reason,modbus_address1,modbus_address2,modbus_address3,run_command,block_reason,gsm_key,green_house_id
8,18345902,6345000000001,2021-04-22 08:59:25,Open,60,99616269,211.199.190.110,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
1066,18802850,6345000000001,2021-04-30 05:52:32,Open,60,99616269,121.154.204.161,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
1218,18867782,6345000000001,2021-05-01 08:48:36,Open,60,99616269,121.154.204.161,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
1230,18897331,6345000000001,2021-05-01 17:39:03,Open,60,99616269,223.39.207.8,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
1244,18932150,6345000000001,2021-05-02 11:37:25,Open,60,99616269,175.202.71.75,NaN,NaN,NaN,NaN,1,NaN,6345,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214440,34459499,6345000000001,2022-10-04 10:22:06,Open,120,AutoControlSystem,NaN,"고온 위험 ,21.3도",NaN,NaN,NaN,1,NaN,6345,NaN
214442,34459507,6345000000001,2022-10-04 10:22:16,Open,120,AutoControlSystem,NaN,"고온 위험 ,21.1도",NaN,NaN,NaN,1,NaN,6345,NaN
214448,34485653,6345000000001,2022-10-05 08:05:12,Open,120,AutoControlSystem,NaN,"고온 위험 ,21.1도",NaN,NaN,NaN,1,NaN,6345,NaN
214450,34485660,6345000000001,2022-10-05 08:05:26,Open,120,AutoControlSystem,NaN,"고온 위험 ,21.1도",NaN,NaN,NaN,1,NaN,6345,NaN
